In [1]:
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")

# Create a console handler and set its level
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Create a formatter and set it on the console handler
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
console_handler.setFormatter(formatter)

# Add the console handler to the logger
logger.addHandler(console_handler)


def main():
    logger.info("Cloud function starting.")
    # 1. Scrape coingecko gainers.
    import scraping.scraper

    scraping.scraper.main()

    # 2. Get contract address for gainers that have them, with coingecko.
    import scraping.contract_addresses

    scraping.contract_addresses.main()

    # 3. Run SQL query on Google Big Query to find wallets that have receieved these tokens within the last 7 days.
    import gbq.gbq

    gbq.gbq.main()


main()


2023-04-03 16:54:30,837 - INFO - Cloud function starting.
2023-04-03 16:54:32,095 - INFO - Scraping complete.
2023-04-03 16:54:32,979 - INFO - Scraping 5 coins that are missing symbols.
2023-04-03 16:54:36,570 - INFO - Scraped https://www.coingecko.com/en/coins/poolz-finance-old
2023-04-03 16:54:40,567 - INFO - Scraped https://www.coingecko.com/en/coins/etho-protocol
2023-04-03 16:54:44,298 - INFO - Scraped https://www.coingecko.com/en/coins/sxp
2023-04-03 16:54:47,977 - INFO - Scraped https://www.coingecko.com/en/coins/injective
2023-04-03 16:54:51,685 - INFO - Scraped https://www.coingecko.com/en/coins/panther-protocol
2023-04-03 16:54:51,718 - INFO - Number of coins with ethereum contract addresses: 35
2023-04-03 16:54:51,718 - INFO - Done scraping contract addresses!
2023-04-03 16:54:51,724 - INFO - Checking 35 token contracts.
2023-04-03 16:54:51,889 - INFO - Running SQL query on Google Big Cloud.
2023-04-03 16:54:55,204 - INFO - Query will process 768 MB.
2023-04-03 16:54:55,206 

In [3]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv
import requests
from google.cloud import bigquery
from google.oauth2 import service_account
import google.auth.transport.requests
import google.oauth2.id_token
import json

load_dotenv()
KEY_PATH = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
GOOGLE_CLOUD_FUNCTION = os.getenv("GOOGLE_CLOUD_FUNCTION")
endpoint = GOOGLE_CLOUD_FUNCTION
audience = endpoint

auth_req = google.auth.transport.requests.Request()
id_token = google.oauth2.id_token.fetch_id_token(auth_req, audience)

# Add message in request args
headers = {"Authorization": f"Bearer {id_token}", "Content-Type": "application/json"}


r = requests.post(
    endpoint, data=json.dumps({"message": "wow json data"}), headers=headers
)

In [5]:
r

<Response [200]>